<a href="https://colab.research.google.com/github/DSF-Kotaro/blog/blob/main/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E6%BC%94%E7%BF%92%E2%85%A0B%E3%80%8010_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>１．株価の時間による変化を確認
<br>２．日ごとの変動を可視化
<br>３．移動平均を計算する
<br>４．複数の株価の終値の相関を計算する
<br>５．複数の株価の変動の関係を見る
<br>６．特定の株のリスクを計算する
<br>７．シミュレーションを使った未来の予測

##株価の時間による変化を確認

In [ ]:
!pip install pandas-datareader

In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import pandas_datareader.data as web
from datetime import datetime

In [2]:
tech_list = ['AAPL','GOOG','MSFT','AMZN']

In [3]:
end = datetime.now()
start = datetime(end.year -1, end.month, end.day)

In [5]:
for stock in tech_list:
  globals()[stock] = web.DataReader(stock, data_source='stooq',start=start,end=end)


In [ ]:
AAPL.describe()

In [ ]:
AAPL.info()

##日ごとの変動を可視化

In [ ]:
AAPL['Close'].plot(legend = True, figsize = (10,4))

In [ ]:
AAPL['Volume'].plot(legend = True, figsize = (10,4))

##移動平均を計算する

In [ ]:
ma_day = [10,20,50]

for ma in ma_day:
  columns_name = 'MA{}'.format(ma)
  AAPL[columns_name] = AAPL['Close'].rolling(ma).mean()

AAPL.head()

In [ ]:
AAPL[['Close','MA10','MA20','MA50']].plot(subplots = False, figsize=(10,4))

##複数の株価の変動の関係を見る

In [ ]:
AAPL['Daily Return'] = AAPL['Close'].pct_change()
AAPL.head()

In [ ]:
AAPL['Daily Return'].plot(figsize = (10,4), legend = True, linestyle ='--', marker ='o')

In [ ]:
sns.distplot(AAPL['Daily Return'].dropna(),bins = 100, color = 'purple')

##複数の株価の終値の相関を計算する

In [ ]:
AAPL_close =AAPL.Close
GOOG_close =GOOG.Close
MSFT_close =MSFT.Close
AMZN_close =AMZN.Close

closing_df = pd.concat([AAPL_close,GOOG_close,MSFT_close,AMZN_close],axis = 1)
closing_df.columns = ['AAPL','GOOG','MSFT','AMZN']
closing_df.head()

In [ ]:
tech_rets = closing_df.pct_change()
tech_rets.head()

In [ ]:
sns.jointplot('GOOG','GOOG',tech_rets,kind = 'scatter', color='seagreen')

In [ ]:
sns.jointplot('GOOG','MSFT',tech_rets,kind = 'scatter', color='seagreen')

In [ ]:
sns.jointplot('AAPL','AMZN',tech_rets,kind = 'scatter', color='seagreen')

In [ ]:
from IPython.display import SVG
SVG(url='http://upload.wikimedia.org/wikipedia/commons/d/d4/Correlation_examples2.svg')

In [ ]:
sns.pairplot(tech_rets.dropna())

In [ ]:
returns_fig = sns.PairGrid(tech_rets.dropna())
returns_fig.map_upper(plt.scatter, color = 'purple')
returns_fig.map_lower(sns.kdeplot, cmap = 'cool_d')
returns_fig.map_diag(plt.hist, bins = 30)

In [ ]:
returns_fig = sns.PairGrid(closing_df.dropna())
returns_fig.map_upper(plt.scatter, color = 'purple')
returns_fig.map_lower(sns.kdeplot, cmap = 'cool_d')
returns_fig.map_diag(plt.hist, bins = 30)

In [ ]:
sns.heatmap(tech_rets.corr(), annot= True)

##特定の株のリスクを計算する

In [ ]:
rets = tech_rets.dropna()
rets.head()

In [ ]:
plt.scatter(rets.mean(),rets.std(),alpha= 0.5, s= np.pi*20)
plt.ylim([0.02,0.03])
plt.xlim([-0.01,0.01])
plt.xlabel('Expected returns')
plt.ylabel('Risk')

for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
  plt.annotate(label,xy = (x, y), xytext = (0, 20),
               textcoords = 'offset points', ha = 'right',
               arrowprops = dict(arrowstyle='-', connectionstyle= 'arc3'))

In [ ]:
sns.distplot(AAPL['Daily Return'].dropna(), bins = 100, color = 'purple')

In [ ]:
rets['AAPL'].quantile(0.05)

##シミュレーションを使った未来の予測

In [32]:
days = 365
dt = 1/days
mu = rets.mean()['GOOG']
sigma = rets.std()['GOOG']

In [33]:
def stock_monte_carlo(start_price,days,mu,sigma):
  price = np.zeros(days)
  price[0] = start_price
  shock = np.zeros(days)
  drift = np.zeros(days)

  for x in range(1,days):
    shock[x] = np.random.normal(loc = mu * dt, scale = sigma * np.sqrt(dt))
    drift[x] = mu * dt
    price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))
  return price

In [ ]:
GOOG.head()

In [ ]:
start_price = GOOG.iloc[0,3]

for run in range(5):
  plt.plot(stock_monte_carlo(start_price,days,mu,sigma))

plt.xlabel('Days')
plt.ylabel('Price')
plt.title('Monte Carlo Analysis')

In [36]:
runs = 10000
simulations = np.zeros(runs)
np.set_printoptions(threshold=5)

for run in range(runs):
    simulations[run] = stock_monte_carlo(start_price,days,mu,sigma)[days-1]


In [ ]:
plt.hist(simulations, bins = 200)

In [ ]:
q = np.percentile(simulations , 1)
plt.hist(simulations,bins = 200)

plt.figtext(0.6,0.8, s = 'Start price:{:0.2f}'.format(start_price))
plt.figtext(0.6,0.7,'mean final price:{:0.2f}'.format(simulations.mean()))
plt.figtext(0.6,0.6,'VaR(0.99):{:0.2}'.format(start_price-q))
plt.figtext(0.15,0.6,'q(0.99):{:0.2f}'.format(q))

plt.axvline(x = q, linewidth = 4, color = 'r')
plt.axvline(x = start_price, linewidth = 4, color = 'g')
plt.axvline(x = simulations.mean(), linewidth = 4, color = 'b')